Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

In [5]:
import json

In [6]:
user_folders = []
d = {
    "FolderName": "Inbox"
}
user_folders.append(d)
d = {
    "FolderName": "Sent Items"
}
user_folders.append(d)

user_folders


In [ ]:
# replace this list with your list of users 
lst_users = [['adelev@xxx' , '11111111-1111-1111-1111-111111111111'], 
            ['alexw@xxx' , '11111111-1111-1111-1111-111111111111']]

for user in lst_users: 
    # print(user)
    user_entry = {
    "initialload_completion_flag": 0,
    "email":user[0],
    "user_id": user[1],
    "id": user[1],
    "displayname":user[0].split('@')[0],
    "region":'West US',
    "folders":user_folders
    }

    json_model = json.dumps(user_entry) 

    # convert to a dataframe
    json_list = []
    json_list.append(json_model)
    df = spark.read.json(sc.parallelize(json_list))
    df.write.format("cosmos.oltp")\
    .option("spark.synapse.linkedService", "CosmosDB")\
    .option("spark.cosmos.container", "users")\
    .mode('append')\
    .save()

 # Uncomment this if your Linked Service is enabled with a private endpoint 
    #df = spark.read.json(sc.parallelize(json_list))
    #df.write.format("cosmos.oltp")\
    #.option("spark.cosmos.useGatewayMode", True)\ # You can uncomment this line if your Linked 
    #.option("spark.synapse.linkedService", "CosmosDb")\
    #.option("spark.cosmos.container", "user")\
    #.mode('append')\
    #.save()

In [9]:
df = spark.read.format("cosmos.oltp")\
            .option("spark.synapse.linkedService", "CosmosDB")\
            .option("spark.cosmos.container", "users")\
            .load()

# Uncomment this if your Linked Service is enabled with a private endpoint 
#df = spark.read.format("cosmos.oltp")\
#            .option("spark.cosmos.useGatewayMode", True)\
#            .option("spark.synapse.linkedService", "CosmosDB")\
#            .option("spark.cosmos.container", "users")\
#            .load()
display(df)            

In [10]:
from pyspark.sql.functions import *


df_folders = df.select('email','folders').select(explode(col("folders")).alias("folders"),'email') \
               .select(["email",col("folders.FolderName").alias("FolderName")])
display(df_folders)